In [10]:
import tkinter as tk
from tkinter import scrolledtext
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import datetime

class SentimentChatbotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sentiment Chatbot")
        self.root.geometry("550x550")
        self.root.configure(bg="#e6f2ff")

        self.chat_display = scrolledtext.ScrolledText(root, state='disabled', wrap='word', bg="#f9f9f9", font=("Segoe UI", 10))
        self.chat_display.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        self.entry = tk.Entry(root, width=80, font=("Segoe UI", 10))
        self.entry.pack(padx=10, pady=(0, 10), side=tk.LEFT, fill=tk.X, expand=True)
        self.entry.bind("<Return>", self.send_message)

        self.send_button = tk.Button(root, text="Send", command=self.send_message)
        self.send_button.pack(padx=(0, 10), pady=(0, 10), side=tk.RIGHT)

        self.sentiment_history = []
        self.display_bot_message("Hi! I’m your offline chatbot. How are you feeling today?")

        # Open log file in append mode
        self.log_file = open("chat_log.txt", "a", encoding="utf-8")
        self.log(f"--- Chat session started at {datetime.datetime.now()} ---\n")

    def log(self, message):
        self.log_file.write(message + "\n")
        self.log_file.flush()

    def analyze_sentiment(self, text):
        blob = TextBlob(text, analyzer=NaiveBayesAnalyzer())
        classification = blob.sentiment.classification
        self.sentiment_history.append(classification)
        if len(self.sentiment_history) > 10:
            self.sentiment_history.pop(0)
        return classification

    def detect_emotion(self, text):
        text = text.lower()
        emotions = {
            'happy': ['happy', 'joy', 'glad', 'excited', 'great', 'fantastic', 'good'],
            'sad': ['sad', 'down', 'unhappy', 'depressed', 'miserable', 'blue'],
            'angry': ['angry', 'mad', 'frustrated', 'annoyed', 'upset', 'hate'],
            'tired': ['tired', 'exhausted', 'sleepy', 'fatigued', 'drained'],
            'stress': ['stress', 'stressed', 'anxious', 'nervous', 'worried', 'overwhelmed'],
            'neutral': []
        }
        for emo, keywords in emotions.items():
            if any(word in text for word in keywords):
                return emo
        return 'neutral'

    def generate_response(self, classification, emotion):
        responses = {
            'happy': [
                "That's wonderful! Keep smiling 😊",
                "I'm glad you're feeling good today!",
                "Awesome! Anything exciting you'd like to share?"
            ],
            'sad': [
                "I'm sorry you're feeling down. Want to talk about it?",
                "It's okay to have tough days. I'm here to listen.",
                "If you want to share more, I'm here for you."
            ],
            'angry': [
                "I understand you're upset. Take a deep breath.",
                "Anger can be tough. Want to try calming exercises?",
                "Let me know if talking about it helps."
            ],
            'tired': [
                "Sounds like you need some rest. Don't forget to take breaks!",
                "Try to relax and recharge when you can.",
                "Rest is important. Make sure to listen to your body."
            ],
            'stress': [
                "Stress can be overwhelming. Have you tried mindfulness?",
                "Taking a short walk can sometimes help relieve stress.",
                "Remember, you’re not alone with this."
            ],
            'neutral': {
                'pos': [
                    "That's nice! What else is on your mind?",
                    "Thanks for sharing! Anything new today?"
                ],
                'neg': [
                    "I hear you. Want to talk more about it?",
                    "Thanks for sharing. I'm here if you need me."
                ]
            }
        }
        if emotion != 'neutral':
            return responses[emotion][len(self.sentiment_history) % len(responses[emotion])]
        else:
            return responses['neutral'][classification][len(self.sentiment_history) % len(responses['neutral'][classification])]

    def display_bot_message(self, message, sentiment='neutral'):
        self.chat_display.configure(state='normal')
        color = {'pos': 'green', 'neg': 'red', 'neutral': 'black'}.get(sentiment, 'black')
        self.chat_display.insert(tk.END, "Bot: " + message + "\n", sentiment)
        self.chat_display.tag_config(sentiment, foreground=color)
        self.chat_display.configure(state='disabled')
        self.chat_display.yview(tk.END)

    def display_user_message(self, message):
        self.chat_display.configure(state='normal')
        self.chat_display.insert(tk.END, "You: " + message + "\n", "user")
        self.chat_display.tag_config("user", foreground="blue")
        self.chat_display.configure(state='disabled')
        self.chat_display.yview(tk.END)

    def send_message(self, event=None):
        user_input = self.entry.get().strip()
        if not user_input:
            return
        self.display_user_message(user_input)
        self.log(f"You: {user_input}")
        self.entry.delete(0, tk.END)

        sentiment = self.analyze_sentiment(user_input)
        emotion = self.detect_emotion(user_input)
        response = self.generate_response(sentiment, emotion)
        self.display_bot_message(response, sentiment)
        self.log(f"Bot: {response}")

    def __del__(self):
        try:
            self.log_file.write(f"\n--- Chat session ended at {datetime.datetime.now()} ---\n\n")
            self.log_file.close()
        except:
            pass

if __name__ == "__main__":
    root = tk.Tk()
    app = SentimentChatbotApp(root)
    root.mainloop()
